In [2]:
!pip install snowflake-snowpark-python==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 1.8 MB/s eta 0:00:0000:01
  Attempting uninstall: asn1crypto
    Found existing installation: asn1crypto 1.4.0
    Uninstalling asn1crypto-1.4.0:
      Successfully uninstalled asn1crypto-1.4.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: filelock
    Found existing installation: filelock 3.3.1
    Uninstalling fi

In [3]:
!pip install cachetools

In [32]:
import os
from pprint import pprint
import re
import datetime
import io
import joblib
from joblib import dump

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Snowflake Connection

In [33]:
from snowflake.snowpark import (
    Column,
    DataFrame,
    Session,
    Window
)

import snowflake.snowpark
from snowflake.snowpark import functions as f
from snowflake.snowpark.types import IntegerType, StringType, StructType, DateType, StructField, MapType

In [34]:
connection_parameters = {
"account": "gi02106.eu-west-2.aws",
"user": "pujaverma",
"password": "Itzme#123",
"role": "accountadmin",
"warehouse": "workshopwh",
"database": "workshopdb",
"schema": "workshopsch",
}

In [35]:
snowflake_conn_session = Session.builder.configs(connection_parameters).create()

In [36]:
snowflake_conn_session.sql("select current_warehouse(), current_database(), current_schema()").show()

---------------------------------------------------------------------
|"CURRENT_WAREHOUSE()"  |"CURRENT_DATABASE()"  |"CURRENT_SCHEMA()"  |
---------------------------------------------------------------------
|WORKSHOPWH             |WORKSHOPDB            |WORKSHOPSCH         |
---------------------------------------------------------------------



In [37]:
snow_df = snowflake_conn_session.table("Snowpark_dataset")

In [38]:
snow_df.schema.fields

[StructField('CHECKING_STATUS', StringType(), nullable=True),
 StructField('DURATION', LongType(), nullable=True),
 StructField('CREDIT_HISTORY', StringType(), nullable=True),
 StructField('PURPOSE', StringType(), nullable=True),
 StructField('CREDIT_AMOUNT', LongType(), nullable=True),
 StructField('SAVINGS_STATUS', StringType(), nullable=True),
 StructField('EMPLOYMENT', StringType(), nullable=True),
 StructField('INSTALLMENT_COMMITMENT', LongType(), nullable=True),
 StructField('PERSONAL_STATUS', StringType(), nullable=True),
 StructField('THER_PARTIES', StringType(), nullable=True),
 StructField('RESIDENCE_SINCE', LongType(), nullable=True),
 StructField('PROPERTY_MAGNITUDE', StringType(), nullable=True),
 StructField('AGE', StringType(), nullable=True),
 StructField('OTHER_PAYMENT_PLANS', StringType(), nullable=True),
 StructField('HOUSING', StringType(), nullable=True),
 StructField('EXISTING_CREDITS', LongType(), nullable=True),
 StructField('JOB', StringType(), nullable=True),


In [39]:
snow_df.show(2)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CHECKING_STATUS"  |"DURATION"  |"CREDIT_HISTORY"                |"PURPOSE"  |"CREDIT_AMOUNT"  |"SAVINGS_STATUS"  |"EMPLOYMENT"  |"INSTALLMENT_COMMITMENT"  |"PERSONAL_STATUS"   |"THER_PARTIES"  |"RESIDENCE_SINCE"  |"PROPERTY_MAGNITUDE"  |"AGE"  |"OTHER_PAYMENT_PLANS"  |"HOUSING"  |"EXISTING_CREDITS"  |"JOB"    |"NUM_DEPENDENTS"  |"OWN_TELEPHONE"  |"FOREIGN_WORKER"  |"CLASS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
snow_df = snow_df.with_column('TARGET', f.when(f.col('CLASS') == 'good',1).otherwise(0))
snow_df = snow_df.drop(f.col('CLASS'))

snow_df.show(2)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CHECKING_STATUS"  |"DURATION"  |"CREDIT_HISTORY"                |"PURPOSE"  |"CREDIT_AMOUNT"  |"SAVINGS_STATUS"  |"EMPLOYMENT"  |"INSTALLMENT_COMMITMENT"  |"PERSONAL_STATUS"   |"THER_PARTIES"  |"RESIDENCE_SINCE"  |"PROPERTY_MAGNITUDE"  |"AGE"  |"OTHER_PAYMENT_PLANS"  |"HOUSING"  |"EXISTING_CREDITS"  |"JOB"    |"NUM_DEPENDENTS"  |"OWN_TELEPHONE"  |"FOREIGN_WORKER"  |"TARGET"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
snow_df_train, snow_df_inf = snow_df.random_split([0.8, 0.2], seed=1234)

snow_df_train.write.save_as_table('training_table', mode="overwrite", create_temp_table=False)
snow_df_inf.write.save_as_table('inference_table', mode="overwrite", create_temp_table=False)

In [44]:
session = Session.builder.configs(connection_parameters).create()
session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools')

def save_file(session, model, path, dest_filename):
    input_stream = io.BytesIO()
    joblib.dump(model, input_stream)
    session._conn.upload_stream(input_stream, path, dest_filename)
    return "successfully created file: " + path

def train_model(session: Session) -> str:
    
    
    df_train = session.table('training_table')

    df_train_pd = df_train.to_pandas()
    
    numerical_cols = list(df_train_pd.select_dtypes(['float64', 'int64']).columns)
    categorical_cols = list(df_train_pd.drop('TARGET', axis=1).select_dtypes(['object']).columns)

    X = df_train_pd.drop('TARGET', axis=1)
    y = df_train_pd['TARGET']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('one_hot', OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)])

    rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier',  RandomForestClassifier(class_weight='balanced',
                                                                 random_state=0))])

    rf_clf = rf_pipe.fit(X_train, y_train)
    
    save_file(session, rf_clf, "@SNOWFLAKESTAGE", 'credit_g_model_2.joblib')

    y_pred = rf_clf.predict(X_test)
    return classification_report(y_test, y_pred)

train_model_sp = f.sproc(train_model, replace=True, session=session)

train_model_sp(session=session)

'              precision    recall  f1-score   support\n\n           0       0.58      0.28      0.38        50\n           1       0.74      0.91      0.81       110\n\n    accuracy                           0.71       160\n   macro avg       0.66      0.59      0.60       160\nweighted avg       0.69      0.71      0.68       160\n'

In [45]:
from snowflake.snowpark.functions import udf
session.add_import("@SNOWFLAKESTAGE/credit_g_model_2.joblib")

In [46]:
query = "create or replace stage udf" +\
        " copy_options = (on_error='skip_file')"
        
session.sql(query).collect()

[Row(status='Stage area UDF successfully created.')]

In [48]:
import cachetools
import joblib
import sys
import os

import pandas as pd

from snowflake.snowpark.functions import udf

@cachetools.cached(cache={})
def read_file(filename):
    import_dir = sys._xoptions.get("snowflake_import_directory")
    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
    return m

features = ['CHECKING_STATUS', 'DURATION', 'CREDIT_HISTORY', 'PURPOSE','CREDIT_AMOUNT', 
            'SAVINGS_STATUS', 'EMPLOYMENT','INSTALLMENT_COMMITMENT', 'PERSONAL_STATUS', 
            'THER_PARTIES', 'RESIDENCE_SINCE', 'PROPERTY_MAGNITUDE', 'AGE', 
            'OTHER_PAYMENT_PLANS', 'HOUSING', 'EXISTING_CREDITS', 'JOB', 'NUM_DEPENDENTS',
            'OWN_TELEPHONE','FOREIGN_WORKER']

@udf(name="predict", is_permanent=True, stage_location="@udf", replace=True, session=session)

def predict(CHECKING_STATUS: str,
            DURATION: int,
            CREDIT_HISTORY: str,
            PURPOSE: str,
            CREDIT_AMOUNT: float,
            SAVINGS_STATUS: str,
            EMPLOYMENT: str,
            INSTALLMENT_COMMITMENT: float,
            PERSONAL_STATUS: str,
            THER_PARTIES: str,
            RESIDENCE_SINCE: float,
            PROPERTY_MAGNITUDE: str,
            AGE: str,
            OTHER_PAYMENT_PLANS: str,
            HOUSING: str,
            EXISTING_CREDITS: float,
            JOB: str,
            NUM_DEPENDENTS: float,
            OWN_TELEPHONE: str,
            FOREIGN_WORKER: str) -> float:
    model = read_file('credit_g_model_2.joblib')
    row = pd.DataFrame([locals()], columns=features)
    return model.predict(row)[0]

In [49]:
from snowflake.snowpark.functions import col

inf_table = 'inference_table'

snow_df_test = session.table(inf_table)
snow_df_test.show(2)
inputs = snow_df_test.drop("TARGET")
inputs.show(2)
snow_df_results = snow_df_test.select(*inputs,
                                      predict(*inputs).alias('PREDICTION'),
                                      (col('TARGET')).alias('ACTUAL_LABEL'))
#snow_df_results.to_pandas().head(20)
snow_df_results.write.mode("overwrite").save_as_table("CREDIT_G_PREDICTIONS")

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CHECKING_STATUS"  |"DURATION"  |"CREDIT_HISTORY"  |"PURPOSE"  |"CREDIT_AMOUNT"  |"SAVINGS_STATUS"  |"EMPLOYMENT"  |"INSTALLMENT_COMMITMENT"  |"PERSONAL_STATUS"   |"THER_PARTIES"  |"RESIDENCE_SINCE"  |"PROPERTY_MAGNITUDE"  |"AGE"  |"OTHER_PAYMENT_PLANS"  |"HOUSING"  |"EXISTING_CREDITS"  |"JOB"               |"NUM_DEPENDENTS"  |"OWN_TELEPHONE"  |"FOREIGN_WORKER"  |"TARGET"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------